# Cеточные модели уравнений с частными производными

## Лабораторная №4 
## (Решение задачи Коши для ОДУ второго и более высоких порядков)

Выполнил Мусатов Данила Юрьевич КМБО-03-18

Задача Коши для ОДУ $n$-го порядка ставится следующим образом:

$$
\begin{aligned}
&y^{(n)}=f\left(x, y, y^{\prime}, y^{\prime \prime}, \ldots, y^{(n-1)}\right) \\
&y\left(x_{0}\right)=y_{0} \\
&y^{\prime}\left(x_{0}\right)=y_{01} \\
&y^{\prime \prime}\left(x_{0}\right)=y_{02} \\
&\cdots \\
&y^{(n-1)}\left(x_{0}\right)=y_{0(n-1)}
\end{aligned}
$$

Основной прием, используемый при решении задач такого типа заключается в введении новых переменных и сведении исходной задачи для ОДУ высокого порядка к решению системы ОДУ первого порядка. Введем новые переменные

$$
\begin{aligned}
&z_{1}=y^{\prime} \\
&z_{2}=y^{\prime \prime} \\
&\cdots \\
&x_{n-1}=y^{(n-1)}
\end{aligned}
$$

$\mathrm{C}$ ними исходную задачу можно переписать в виде системы $n$ ОДУ пер вого порядка:

$$
\left\{\begin{array}{l}
y^{\prime}=z_{1}, \\
z_{1}^{\prime}=z_{2}, \\
z_{2}^{\prime}=z_{3}, \\
\cdots \\
z_{n-2}^{\prime}=z_{n-1}, \\
z_{n-1}^{\prime}=f\left(x, y, z_{1}, \ldots, z_{n-1}\right)
\end{array}\right.
$$


### Задание:
Решить задачу Коши со значениями шага $h_{1}=0.1$ и $h_{2}=0.01 .$ Сравнить результаты графически.
#### Вариант №2: 
$$
\begin{aligned}
x^{4} y^{\prime \prime}+2 x^{3} y^{\prime}+y=0 \quad\qquad\\
y(1)=1, \quad y^{\prime}(1)=2, \quad x \in[1,2]
\end{aligned}
$$

### Решение заменой

$$
\left\{\begin{array}{l}
x^{4}z^{\prime}+2 x^{3}z+y=0 \\
z=y^{\prime}\\
y(1)=1 \\ 
y^{\prime}(1)=2 \\ 
x \in[1,2]
\end{array}\right.\qquad \qquad\qquad
\left\{\begin{array}{l}
z^{\prime}=-\dfrac{2x^{3}z+y}{x^{4}} \\
z=y^{\prime}\\
y(1)=1 \\ 
z(1)=2 \\ 
x \in[1,2]
\end{array}\right.
$$

### Решение аналитическое
$$
\begin{aligned}
y(x)=\cos\left(\dfrac{x-1}{x}\right)+2\sin\left(\dfrac{x-1}{x}\right)
\end{aligned}
$$

In [1]:
import matplotlib.pyplot as plt
#from math import cos, sin
from IPython.display import display
import ipywidgets as ip_w 
import numpy as np

func = lambda x, y, z: (- ((2*(x**3))*z) - y)/(x**4)

source_func = lambda x:  (2*np.sin((x - 1)/x) + np.cos((x - 1)/x))    # аналитическое решение

# Функция для оценки максимального отклонения от аналитического решения
def max_error(xs, ys, source_func):
    return max(np.fabs(ys - source_func(xs)))

def Method_Euler_without_recalc(func, y0, z0, a, b, h):
    #Euler method withot recalc
    xs = np.arange(a, b + h, h)
    ys = [y0]
    zs = [z0]
    for x in xs[1:]:
        ys.append(ys[-1] + h*zs[-1])
        zs.append(zs[-1] + h*func(x-h,ys[-1], zs[-1]))
    return xs, np.array(ys) , np.array(zs) 

def source_without_recalc(y0, z0, a, b, h): #сравнение между методом Эйлера и точным значением
    xs, ys, _ = Method_Euler_without_recalc(func, y0, z0, a, b, h)
    plt.rcParams["figure.figsize"] = (15,8)
    plt.grid(True)
    plt.plot(xs, ys, color="blue", label="Метод эйлера без перерасчета с шагом " + str(h))
    plt.plot(xs, source_func(xs), color="green", label=f"Точное аналитическое решение Задачи c шагом " + str(h))
    plt.legend()
    plt.show()
    print("Максимальное отклонение от аналитического решения с шагом " + str(h), "равно : ", max_error(xs, ys, source_func))

In [2]:
Y0=1
Z0=2
A=1
B=2

In [3]:
ip_w.interact(source_without_recalc,  y0 = ip_w.fixed(Y0), z0 = ip_w.fixed(Z0), a=ip_w.fixed(A), b=ip_w.fixed(B), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

## Оценка погрешности с помощью правила Рунге

$$error = \frac{| y_h - y_{h/2}|}{2^p - 1}$$ 

In [4]:
def Runge_rule(ys1, ys2, p=1):
    return max(np.fabs(ys1 - ys2))/(2**p - 1)

Для метода Эйлера без перерасчета порядок точности равен 1

In [5]:
def Euler_estimation_error_Ruge_depending_on_h(y0,z0,a,b,h):
    _, ys1, _ = Method_Euler_without_recalc(func, y0,z0, a, b, h)
    _, ys2,_ = Method_Euler_without_recalc(func, y0, z0, a, b, h/2)
    if(len(ys1)!=len(ys2[::2])):
        ys1=np.delete(ys1,len(ys1)-1,0)
    err1 = Runge_rule(ys1, ys2[::2])
    print(f" Ошибка по Правило рунге для шага {h} равна {round(err1, 7)}" )

In [6]:
ip_w.interact(Euler_estimation_error_Ruge_depending_on_h,  y0 = ip_w.fixed(Y0), z0 = ip_w.fixed(Z0), a=ip_w.fixed(A), b=ip_w.fixed(B), p=ip_w.fixed(1), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

Ошибка по Правило рунге для шага 0.01 равна 0.0016893 </br>
Ошибка по Правило рунге для шага 0.1 равна 0.0200142

In [7]:
from scipy.integrate import odeint as ODE

In [8]:
def f_m(y_m,x):
    y = y_m[0]
    dy = y_m[1]  
    ydot = [[],[]]
    ydot[0] = dy
    ydot[1] = (- (2*x**3)*dy - y)/(x**4)
    return ydot

def scipy_ODE(func, y_m, a, b, h):
    xs = np.arange(a, b+h, h)
    ys = ODE(func,y_m,xs)
    return xs, ys

In [9]:
def comparison_source_with_Scipy(y0, z0, a, b, h):
    y_m=[y0,z0]
    xs, ys = scipy_ODE(f_m, y_m, a, b, h)
    plt.rcParams["figure.figsize"] = (15,8)
    plt.grid(True)
    plt.plot(xs, ys[:,0], color="blue", label="Решение с помощью модуля Scipy " + str(h))
    plt.plot(xs, source_func(xs), color="green", label=f"Точное аналитическое решение Задачи c шагом " + str(h))
    plt.legend()
    plt.show()    
    print("Максимальное отклонение от аналитического решения с шагом " + str(h), "равно : ", max_error(xs, ys[:,0], source_func))


In [10]:
ip_w.interact(comparison_source_with_Scipy,  y0 = ip_w.fixed(Y0), z0 = ip_w.fixed(Z0), a=ip_w.fixed(A), b=ip_w.fixed(B), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

Значения от аналитического решения отличаются на 10^(-8)